<a href="https://colab.research.google.com/github/zyuneee/portfolio/blob/main/cat_vs_dog_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content'
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

In [ ]:
!unzip -q dogs-vs-cats-redux-kernels-edition.zip -d .config/

In [ ]:
import tensorflow as tf

print(os.listdir('/content/.config/train'))


In [ ]:
!unzip -q /content/.config/train.zip -d .config/

In [ ]:
# 데이터 폴더별로 분류
# 고양이는 cat 폴더로, 강아지는 dog 폴더로 가라


#파이썬으로 파일만들기
os.mkdir('/content/dataset/')
os.mkdir('/content/dataset/cat')
os.mkdir('/content/dataset/dog')


import shutil
for i in os.listdir('/content/.config/train/'):
  if 'cat'in i:
    shutil.copyfile('/content/.config/train/'+i,'/content/dataset/cat/'+i )
  if 'dog'in i:
    shutil.copyfile('/content/.config/train/'+i,'/content/dataset/dog/'+i )



In [2]:
import tensorflow as tf

train_ds=tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(64,64), #이미지 사이즈 통일
    batch_size=64,
    subset='training',
    validation_split=0.2,
    seed=1234,
)

val_ds=tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(64,64), #이미지 사이즈 통일
    batch_size=64,
    subset='validation',
    validation_split=0.2,
    seed=1234
)

#이미지데이터 크기가 크면 학습에 오래걸리므로 모든 값을 0~1사이로 압축하는 게 좋음

def 전처리함수(i,정답):
  i=tf.cast(i/255.0,tf.float32)
  return i,정답

#.map=모든 i에 전처리함수 적용해라

train_ds = train_ds.map(전처리함수)
val_ds=val_ds.map(전처리함수)

#preprocessing메서드 쓰면,( (x,x,x,x,), (y,y,y,y,y,)) 이런 형식으로 됨

#결과, train_ds는 ((이미지2만개),(정답2만개))vs val_ds는 ((이미지5천개),(정답5천개)) 요런형태로 있음있음

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [3]:
#  import matplotlib.pyplot as plt

#  for i, 정답 in train_ds.take(1):
#    print(i)
#    print(정답)
# #   plt.imshow(i[0].numpy().astype('uint8')) #텐서를 numpy로 변환시
# #   plt.show()

In [4]:
model= tf.keras.Sequential([
    #tf.keras.layers.Dense(128,input_shape=(28,28),activation='relu'),
    tf.keras.layers.Conv2D(32,(3,3),padding='same', activation='relu',input_shape=(64,64,3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3),padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2),#overfitting 막기위해 사용: 노드들 중 20%제거해주세요
    tf.keras.layers.Conv2D(128,(3,3),padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1,activation='sigmoid'),
])

model.summary()

exit()

#모델 컴파일
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(train_ds,validation_data=val_ds,epochs=1)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,048,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,142,081 (4.36 MB)

 Trainable params: 1,142,081 (4.36 MB)

 Non-trainable params: 0 (0.00 B)

313/313 ━━━━━━━━━━━━━━━━━━━━ 234s 737ms/step - accuracy: 0.5707 - loss: 0.6791 - val_accuracy: 0.7124 - val_loss: 0.5532
